In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.applications.inception_v3 import InceptionV3
#from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

import os
import shutil
import glob
#from keras import backend as K
#K.set_image_dim_ordering('th')

import numpy as np
import pandas as pd
#import h5py

import matplotlib.pyplot as plt

## 1. Препроцессинг изображений

### 1.1 Очистка папок

In [2]:
def clear_folder(mydir):
    filelist = [f for f in os.listdir(mydir)]
    for f in filelist:
        os.remove(os.path.join(mydir, f))

In [3]:
for i in os.listdir('data/train'):
    clear_folder(f'data/train/{i}')
    
for i in os.listdir('data/test'):
    clear_folder(f'data/test/{i}')

### 1.2 Выбор изображений

In [4]:
data_names = list(glob.glob('data/Aldanella_attleborensis/*Plate_?_?*a.tif'))
f = open('data/Aldanella_attleborensis/different_angle.txt', 'rt')
diff_angle = f.readlines()
for i, pic in enumerate(diff_angle):
    diff_angle[i] = f'data/Aldanella_attleborensis\\{pic.rstrip()}.tif'
for pic in diff_angle:
    if pic in data_names:
        data_names.remove(pic)
f.close()
#print(data_names)
for i, n in enumerate(range(len(data_names))):
    if i <= int(len(data_names)*0.75):
        shutil.copy(data_names[n], 'data/train/class_att')
    else:
        shutil.copy(data_names[n], 'data/test/class_att')

In [5]:
data_names = list(glob.glob('data/Aldanella_sibirica_sp_nov/*Plate_?_?*a.tif'))
f = open('data/Aldanella_sibirica_sp_nov/different_angle.txt', 'rt')
diff_angle = f.readlines()
for i, pic in enumerate(diff_angle):
    diff_angle[i] = f'data/Aldanella_sibirica_sp_nov\\{pic.rstrip()}.tif'
for pic in diff_angle:
    if pic in data_names:
        data_names.remove(pic)
f.close()
#print(data_names) 
for i, n in enumerate(range(len(data_names))):
    if i <= int(len(data_names)*0.75):
        shutil.copy(data_names[n], 'data/train/class_sib')
    else:
        shutil.copy(data_names[n], 'data/test/class_sib')

### 1.3 Создание тензеров из изображений

Не будем пока делать аугментацию (повороты, добавления шума) изображений. Сделаем это при тренировки полной модели

In [6]:
datagen = ImageDataGenerator(rescale=1./255) # масштабируем именно значения каждого пикселя

In [7]:
#Пропускаем через свёртучнуя чать сети изображения (как "train", так и "test"), чтобы выделить массивы наиболее важных признаков

train_generator = datagen.flow_from_directory('data/train/', #Важно разложить по папкам объекты классов
                                        target_size=(150, 150), #Размер изображения (высота и ширина в пикселях)
                                        batch_size=10, #количество изображений, пропускаемых через сеть на каждой итерации,
                                        color_mode='rgb',
                                        shuffle=False) #не перемешиваем, т.к. эти данные будем прогонять через уже обученную
                                                       #сеть. Далее будет удобно работать с индексами классов при загрузке
                                                       #массивов numpy для обучения полносвязанных слоёв
                                                       # keep data in same order as labels

        
#Нужно настроить: color_mode=не "rgb" (3 канала), а "grayscale" (1 канал)
#Трансофрмацию валидационных данных не нужно делать
test_generator = datagen.flow_from_directory('data/test/',
                                              target_size=(150, 150), #разобраться с количеством тестовых
                                                                      #нужно, чтобы их было 25%, а не 50%
                                              batch_size=10, #количество изображений, пропускаемых через сеть на каждой итерации,
                                              color_mode='rgb',
                                              shuffle=False)

Found 53 images belonging to 2 classes.
Found 17 images belonging to 2 classes.


## 2. Создание модели

In [8]:
inc_model = InceptionV3(include_top=False, weights='imagenet', pooling='avg', input_shape=((150, 150, 3)))

In [9]:
features_train = inc_model.predict(train_generator) #2000 значит, что мы предскажем 
                                                    #только первые 2000 элементов из папки
np.save(open('data/train_binary/bn_features_train.npy', 'wb'), features_train) #'wb' - открыть/создать
                                                                               #файл для двоичной записи

features_test = inc_model.predict(test_generator) #Настроить количество предсказаний
np.save(open('data/test_binary/bn_features_test.npy', 'wb'), features_test)

2/2 [==============================] - 0s 144ms/step


In [10]:
train_data = np.load(open('data/train_binary/bn_features_train.npy', 'rb'))
train_labels = np.array([0] * 19 + [1] * 34) 

test_data = np.load(open('data/test_binary/bn_features_test.npy', 'rb'))
test_labels = np.array([0] * 6 + [1] * 11)

In [11]:
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data.shape[1:])) #Явно не создаём слой ввода (input)
                                                        #'Flatten' cжимает тензор на входе до вектора заданной формы
fc_model.add(Dense(128, activation='relu', name='dense_one')) #можно попробовать функцию активации 'selu'
fc_model.add(Dropout(0.5, name='dropout_one')) #выбрасываем сигналы от половины нейронов
fc_model.add(Dense(128, activation='relu', name='dense_two'))
fc_model.add(Dropout(0.5, name='dropout_two'))
fc_model.add(Dense(128, activation='relu', name='dense_three'))
fc_model.add(Dropout(0.5, name='dropout_three'))
fc_model.add(Dense(1, activation='sigmoid', name='output')) #использовать 'softmax' для многоклассовой классификации
#количество нейронов в выходном слое равно количеству классов (записываем максимальный номер класса)

fc_model.compile(optimizer='adam', #поиск минимума (может быть 'adam', 'sgd', 'rmsprop')
              loss='binary_crossentropy', # 'categorical_crossentropy'
              metrics=['accuracy'])

In [12]:
fc_model.fit(train_data, train_labels,
            epochs=50, batch_size=10,
            validation_data=(test_data, test_labels))

fc_model.save_weights('data/model_weights/first_test.hdf5') #

Epoch 1/50
6/6 [==============================] - 1s 31ms/step - loss: 1.6787 - accuracy: 0.4906 - val_loss: 0.7396 - val_accuracy: 0.4706
Epoch 2/50
6/6 [==============================] - 0s 8ms/step - loss: 1.5238 - accuracy: 0.5660 - val_loss: 0.6471 - val_accuracy: 0.7059
Epoch 3/50
6/6 [==============================] - 0s 7ms/step - loss: 2.0905 - accuracy: 0.5283 - val_loss: 0.5510 - val_accuracy: 0.5882
Epoch 4/50
6/6 [==============================] - 0s 7ms/step - loss: 1.3566 - accuracy: 0.6604 - val_loss: 0.5265 - val_accuracy: 0.7647
Epoch 5/50
6/6 [==============================] - 0s 7ms/step - loss: 1.2983 - accuracy: 0.5660 - val_loss: 0.4790 - val_accuracy: 0.7647
Epoch 6/50
6/6 [==============================] - 0s 8ms/step - loss: 1.3238 - accuracy: 0.5849 - val_loss: 0.5415 - val_accuracy: 0.7059
Epoch 7/50
6/6 [==============================] - 0s 8ms/step - loss: 0.9378 - accuracy: 0.6981 - val_loss: 0.5845 - val_accuracy: 0.6471
Epoch 8/50
6/6 [=================

In [13]:
#fc_model.evaluate(test_data, test_labels, batch_size=1)

In [14]:
result = fc_model.evaluate(test_data, test_labels)
dict(zip(fc_model.metrics_names, result))

1/1 [==============================] - 0s 18ms/step - loss: 1.1285 - accuracy: 0.7059


{'loss': 1.1284996271133423, 'accuracy': 0.7058823704719543}

In [15]:
x = Flatten()(inc_model.output)
x = Dense(128, activation='relu', name='dense_one')(x)
x = Dropout(0.5, name='dropout_one')(x)
x = Dense(128, activation='relu', name='dense_two')(x)
x = Dropout(0.5, name='dropout_two')(x)
x = Dense(128, activation='relu', name='dense_three')(x)
x = Dropout(0.5, name='dropout_three')(x)
top_model=Dense(1, activation='sigmoid', name='output')(x)
model = Model(inputs=inc_model.input, outputs=top_model)

In [16]:
weights_filename='data/model_weights/first_test.hdf5'
model.load_weights(weights_filename, by_name=True)

In [17]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=1e-4, momentum=0.9),
                #optimizer='adam',
              metrics=['accuracy'])

In [18]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=90)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('data/train/',
                                        target_size=(150, 150),
                                        batch_size=10,
                                        color_mode='rgb',
                                        class_mode='binary',
                                        shuffle=True)

validation_generator = test_datagen.flow_from_directory('data/test/',
                                              target_size=(150, 150),
                                              batch_size=10,
                                              color_mode='rgb',
                                              class_mode='binary',
                                              shuffle=True)

'''
pred_generator=test_datagen.flow_from_directory('data/img_val/',
                                                     target_size=(150,150),
                                                     batch_size=100,
                                                     class_mode='binary')
'''

Found 53 images belonging to 2 classes.
Found 17 images belonging to 2 classes.


"\npred_generator=test_datagen.flow_from_directory('data/img_val/',\n                                                     target_size=(150,150),\n                                                     batch_size=100,\n                                                     class_mode='binary')\n"

In [19]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, #new_lr = lr * factor
                              patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=20)

In [20]:
model.fit(
        train_generator,
        epochs=150,
        validation_data=validation_generator,
        callbacks=[early_stopping]
        )

Epoch 1/150
6/6 [==============================] - 7s 320ms/step - loss: 0.7512 - accuracy: 0.7170 - val_loss: 0.9814 - val_accuracy: 0.7059
Epoch 2/150
6/6 [==============================] - 1s 119ms/step - loss: 0.6819 - accuracy: 0.6415 - val_loss: 0.8579 - val_accuracy: 0.7059
Epoch 3/150
6/6 [==============================] - 1s 110ms/step - loss: 0.6537 - accuracy: 0.6038 - val_loss: 0.7321 - val_accuracy: 0.7059
Epoch 4/150
6/6 [==============================] - 1s 114ms/step - loss: 0.8137 - accuracy: 0.5472 - val_loss: 0.6355 - val_accuracy: 0.7059
Epoch 5/150
6/6 [==============================] - 1s 117ms/step - loss: 0.6580 - accuracy: 0.5849 - val_loss: 0.5420 - val_accuracy: 0.7059
Epoch 6/150
6/6 [==============================] - 1s 111ms/step - loss: 0.5704 - accuracy: 0.6792 - val_loss: 0.4708 - val_accuracy: 0.7059
Epoch 7/150
6/6 [==============================] - 1s 111ms/step - loss: 0.5600 - accuracy: 0.7358 - val_loss: 0.4327 - val_accuracy: 0.7059
Epoch 8/150
6

In [21]:
#model.evaluate_generator(pred_generator)
result = model.evaluate(validation_generator)
dict(zip(fc_model.metrics_names, result))

2/2 [==============================] - 0s 39ms/step - loss: 0.5041 - accuracy: 0.7059


{'loss': 0.5041056871414185, 'accuracy': 0.7058823704719543}